In [1]:
with open ("stu_08.dat", "r") as f:
    
    def mutation(v):
        positions = []
        for place, val in enumerate (v):
            if val.isalpha():
                positions.append(place)
        return positions
    
    found = 0
    
    pat_num = 1
    all_pat_num = []

    mutA_query = []
    all_mutA_query = []
    
    mutB_query = []
    all_mutB_query = []  
    
    for line in f:

        if 'Query_' in line:
            #print (line)
            found = 1
            query = line.split()
            begin = int(query[1]) #get the start position for query
            
        elif line == "\n":
            found = 0
            
        elif found == 1:
            
            if "NODE_60" in line or "NODE_61" in line:
                hit = line.split()
                bases = hit[2]
                mut = mutation(bases)
        
            # Determine which query to adjust the mutation positions for
            if "NODE_60" in line:
                adjusted_positions = [begin + q_pos for q_pos in mut]
                mutA_query.extend(adjusted_positions)
            elif "NODE_61" in line:
                adjusted_positions = [begin + q_pos for q_pos in mut]
                mutB_query.extend(adjusted_positions)
            
        elif "Matrix: blastn matrix 1 -2" in line:
            
            all_mutA_query.append(mutA_query) #collect data for patients
            all_mutB_query.append(mutB_query)
            all_pat_num.append(pat_num)
            
            # resets for the next patient                     
            mutA_query = []
            mutB_query = []  
                
            pat_num+=1
                         
    wanted_mut = [134, 443, 769, 955, 990, 1051, 1078, 1941, 2138, 2638, 3003]
    mutation_names = ['T134A', 'A443G', 'G769C', 'G955C', 'A990C', 'G1051A', 'G1078T', 'T1941A',  'T2138C', 'G2638T', 'A3003T']

    #for table header
    print("\t" + "\t".join(mutation_names))

    # to get the patient number and the corresponding mutations for node 60 and 61 from the lists:
    for patient, patient_mutationsA, patient_mutationsB in zip(all_pat_num, all_mutA_query, all_mutB_query):
        
        common_mutationsA = set(patient_mutationsA) & set(wanted_mut) #made them to sets to find the ones that exist in both
        common_mutationsB = set(patient_mutationsB) & set(wanted_mut)
        
        allele = []
    
        for y in wanted_mut:  
            if y in common_mutationsA and y in common_mutationsB:
                allele.append("2")  
            elif y in common_mutationsA or y in common_mutationsB:
                allele.append("1")   
            else:
                allele.append("0")

        #for table rows
        print(f"pat{patient:02}\t" + "\t".join(allele)) # to print each item of the list in the same line separated by tab

	T134A	A443G	G769C	G955C	A990C	G1051A	G1078T	T1941A	T2138C	G2638T	A3003T
pat01	1	0	1	0	1	1	1	1	0	1	1
pat02	0	0	1	0	1	2	2	0	0	0	1
pat03	1	1	1	0	1	2	2	0	1	0	1
pat04	1	0	1	0	1	1	1	0	0	0	1
pat05	0	0	1	0	1	1	1	1	1	0	1
pat06	1	1	1	0	1	2	2	0	0	1	1
pat07	0	0	0	1	1	2	2	1	0	1	1
pat08	2	2	0	1	1	0	0	2	2	1	1
pat09	0	1	1	1	1	2	2	2	0	0	1
pat10	0	0	1	0	1	2	2	0	0	0	1
pat11	0	0	0	0	0	0	0	0	0	0	1
pat12	1	0	1	1	1	1	1	0	1	0	1
pat13	0	1	1	1	1	1	1	0	0	0	1
pat14	1	0	1	0	1	2	2	0	0	1	1
pat15	0	1	1	1	1	2	2	0	1	0	1
pat16	2	2	0	1	1	0	0	2	2	1	1
pat17	0	1	1	0	1	2	2	0	0	0	1


In [1]:
#Comments:
# I set pat_num=1 outside of the loop and then every time it gets to the last line of the patient it adds 1 to it for the next patient
# I used found=0 approach to select lines after the match as suggested, when it finds Query_ it sets to 1 and when it gets to an empty line it resets to 0
# I identified and adjusted the mutation positions for each haploid like in the easy project and gathered the data for each patient in the first part like suggested
# Then I found which of the wanted mutations were found for each patient in each haploid by using "sets &" to find the mutations that exist in both sets
# Then I check each wanted mutation to see if it was found in one or both haploids of each patient and append the corresponing value to a list
# And then I use f-strings to get the wanted format 